In [1]:
import matplotlib
# matplotlib.use('Qt5Agg')
from matplotlib import pyplot as plt
from matplotlib.widgets import Button
import numpy as np
from skimage.io import imread, imsave
import json
import sys

In [2]:
dmd_dir = r'/groups/ahrens/home/limj2/dmd/' 
# dmd_dir = r'C:\Users\Ahrenslab\Documents\Github\dmd\'

In [3]:
sys.path.append(dmd_dir)
from dmd.gui import DraggableRectangle, RoiDrawing
plt.ion()

In [4]:
def get_sample_image_location(sample_image_fname):
    # if a metadata file is available, load it and get the position of the camera ROI
    from fish.image.zds import get_metadata
    from os.path import split, sep, exists
    from numpy import array
    sample_image_dir = split(sample_image_fname)[0] + sep
    metadata_path = sample_image_dir + 'ch0.xml'
    roi = None
    if exists(metadata_path):
        metadata = get_metadata(metadata_path)
        roi = array(metadata['camera_roi'].split('_')).astype('int')
    return roi

In [5]:
calibration_data_fname = os.path.join(dmd_dir, 'dmd_calibration.json')

In [6]:
calibration_image_fname = os.path.join(dmd_dir, 'target.png') # r'F:\rongwei\cross_hair\cross_hair_20210712.png'    
calibration_image = imread(calibration_image_fname)

## Load calibration image from file and estimate position of DMD pattern 
# todo: automate this

In [ ]:
from matplotlib.patches import Rectangle
fig, axs = plt.subplots(figsize=(8,8))
from skimage.exposure import adjust_gamma
axs.imshow(adjust_gamma(calibration_image,.5), origin='lower', cmap='gray')

# Empirically, this is a pretty good guess, in case we can't load the calibration data file
# x_guess, y_guess, w_guess, h_guess = 230, 588, 1600, 874

with open(calibration_data_fname, 'r') as infile:
    calibration_data = json.load(infile)

x_guess, y_guess, w_guess, h_guess = calibration_data['dmd_rect']

# dmd_rect stores the coordinates of the corners of the rectangle instead of corner_x, corner_y, w, h
w_guess -= x_guess
h_guess -= y_guess

rect = Rectangle((x_guess,y_guess), w_guess, h_guess, fill=False)
axs.add_patch(rect)
dr = DraggableRectangle(rect)
dr.connect()
# plt.show()

In [6]:
## TODO:Add a crosshair down the middle of the alignment rectangle

In [7]:
x, y, w, h = dr.rect.get_x(), dr.rect.get_y(), dr.rect.get_width(), dr.rect.get_height()
dmd_rect = [int(x), int(y), int(x + w), int(h + y)]
calibration_data = {'dmd_rect' : dmd_rect}
with open(calibration_data_fname, 'w') as outfile:
    json.dump(calibration_data, outfile)

print(x,y,w,h)

236 580 1580 894


## Load target image from file and draw ROI

In [8]:
sample_image_fname = r'F:\jing\20210813\fish00\7dpf_gfap-gal4-uas-cochr_elavl3-rgeco_opto-ref-stack_fish00_exp00_20210813_175732\TM0000000_CM0_CHN00.tif'
from skimage.exposure import adjust_gamma

# try to get location of the camera image from metadata
sample_image_pos = get_sample_image_location(sample_image_fname)
# if we can't find the metadata,assume the sample image was taken from (0,512) to (2048,1536)  

sample_image = imread(sample_image_fname)

#special case when sample image is the full chip
if sample_image.shape == (2048,2048):
    sample_image_pos = [0,2048,0,2048]

if sample_image_pos is None:
    # following the zebrascope metadata, this is [y_start, y_end, x_start, x_end]
    sample_image_pos = [512, 1536, 0, 2048]


if sample_image.ndim ==3:
    sample_image = sample_image.max(0)
    
full_cam = np.zeros((2048,2048)) + sample_image.min()
full_cam[sample_image_pos[0]:sample_image_pos[1], sample_image_pos[2]:sample_image_pos[3]] = sample_image
sample_image = full_cam
data = sample_image[dmd_rect[1]:dmd_rect[3],dmd_rect[0]:dmd_rect[2]]

fig = plt.figure()
ax = plt.axes([.05, .05, .6, .6])
ax.set_title('Draw ROI')
ax.imshow(adjust_gamma(data,.1), cmap='gray')
#
roi_draw = RoiDrawing(ax, data)

ax_butt = plt.axes([0.7, 0.05, 0.1, 0.075])
wipe_butt = Button(ax_butt, 'Wipe')
wipe_butt.on_clicked(roi_draw.wipe)

ax_decr = plt.axes([.7, .15, .1, .075])
decr_button = Button(ax_decr, 'Prev ROI')
decr_button.on_clicked(roi_draw.focus_decr)

ax_incr = plt.axes([.7, .25, .1, .075])
incr_button = Button(ax_incr, 'Next ROI')
incr_button.on_clicked(roi_draw.focus_incr)
plt.show()

NameError: name 'sample_image' is not defined

In [10]:
masks = np.array([r.get_mask() for r in roi_draw.rois if len(r.x) > 0])
# apply binary opening to get rid of thin stuff
from skimage.morphology import binary_opening, disk
from skimage.transform import resize
masks = np.array([binary_opening(m, disk(5)) for m in masks])
dmd_dims = (684,608)  ## needs updating
masks_resized = np.array([resize(m, output_shape = dmd_dims, mode='constant', cval=0) for m in masks])
# need to flip along x axis
masks_resized = masks_resized[:,:,::-1]
fig, axs = plt.subplots(ncols=2)

axs[1].imshow(masks_resized.sum(0),cmap='gray', origin='lower')
axs[0].imshow(masks.sum(0), cmap='gray')

plt.show()

In [ ]:
## untested
fig, axs = plt.subplots()
plt.imshow(masks.reshape(2,-1,masks.shape[1], masks.shape[2]).sum(1)[1])
plt.show()

In [11]:
len(masks)

6

## Set output directory for ROI

In [12]:
out_path = r'W:\limj2\dmd\20210813\fish00\TS1-reg1'
from os.path import exists
from os import makedirs
if not exists(out_path):
    makedirs(out_path)

# Warning: this overwrites any existing ROI!!

In [13]:
for ind, m in enumerate(masks_resized):
    imsave(out_path + '\pattern_{:04d}.png'.format(ind), m)

C:\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))
